# grupo6-img

Essa branch é derivada da "grupo6". É aqui onde fica a subclasse para reconhecimento de imagens pelo YOLO.



In [ ]:
from grp6_bot import BotTelegram # Essa linha importa a classe mãe do arquivo grp6_bot.py
# Se você quiser trabalhar com esse arquivo direto no Colab, você precisa jogar esse arquivo nas Files
from ultralytics import YOLO
import tempfile
import os
import numpy as np

# Traduções de todas as classes detectáveis para o português:
TRADUCOES = {
    "person": "pessoa",
    "bicycle": "bicicleta",
    "car": "carro",
    "motorcycle": "moto",
    "airplane": "avião",
    "bus": "ônibus",
    "train": "trem",
    "truck": "caminhão",
    "boat": "barco",
    "traffic light": "semáforo",
    "fire hydrant": "hidrante",
    "stop sign": "placa de pare",
    "parking meter": "parquímetro",
    "bench": "banco",
    "bird": "pássaro",
    "cat": "gato",
    "dog": "cachorro",
    "horse": "cavalo",
    "sheep": "ovelha",
    "cow": "vaca",
    "elephant": "elefante",
    "bear": "urso",
    "zebra": "zebra",
    "giraffe": "girafa",
    "backpack": "mochila",
    "umbrella": "guarda-chuva",
    "handbag": "bolsa",
    "tie": "gravata",
    "suitcase": "mala",
    "frisbee": "frisbee",
    "skis": "esquis",
    "snowboard": "snowboard",
    "sports ball": "bola esportiva",
    "kite": "pipa",
    "baseball bat": "taco de beisebol",
    "baseball glove": "luva de beisebol",
    "skateboard": "skate",
    "surfboard": "prancha de surfe",
    "tennis racket": "raquete de tênis",
    "bottle": "garrafa",
    "wine glass": "taça de vinho",
    "cup": "copo",
    "fork": "garfo",
    "knife": "faca",
    "spoon": "colher",
    "bowl": "tigela",
    "banana": "banana",
    "apple": "maçã",
    "sandwich": "sanduíche",
    "orange": "laranja",
    "broccoli": "brócolis",
    "carrot": "cenoura",
    "hot dog": "cachorro-quente",
    "pizza": "pizza",
    "donut": "rosquinha",
    "cake": "bolo",
    "chair": "cadeira",
    "couch": "sofá",
    "potted plant": "planta em vaso",
    "bed": "cama",
    "dining table": "mesa de jantar",
    "toilet": "vaso sanitário",
    "tv": "televisão",
    "laptop": "notebook",
    "mouse": "mouse",
    "remote": "controle remoto",
    "keyboard": "teclado",
    "cell phone": "celular",
    "microwave": "micro-ondas",
    "oven": "forno",
    "toaster": "torradeira",
    "sink": "pia",
    "refrigerator": "geladeira",
    "book": "livro",
    "clock": "relógio",
    "vase": "vaso",
    "scissors": "tesoura",
    "teddy bear": "urso de pelúcia",
    "hair drier": "secador de cabelo",
    "toothbrush": "escova de dentes"
}

# Criação de uma função auxiliar do YOLO para agrupar detecções por classe,
# remover falsos positivos e obter uma maior confiança final:
def analise_estatistica_yolo(deteccoes):

    """
    Recebe uma lista de detecções no formato:
    [{"classe": str, "conf": float}, ...]

    Retorna uma lista com:
    - Classe detectada
    - Confiança final (em forma de mediana)
    - Número de amostras válidas
    """

    agrupado = {}

    # Agrupa as confianças por classe detectada:
    for d in deteccoes:
        if d["classe"] not in agrupado:
            agrupado[d["classe"]] = []
        agrupado[d["classe"]].append(d["conf"])

    resultado = []

    # Aplica estatística robusta para cada classe:
    for classe, confs in agrupado.items():
        confs = np.array(confs)

        # Quartis para cálculo do IQR (Interquartile Range):
        q1 = np.percentile(confs, 25)
        q3 = np.percentile(confs, 75)
        iqr = q3 - q1

        # Remove valores fora do intervalo aceitável a partir do IQR (outliers):
        confs_validas = confs[
            (confs >= q1 - 1.5 * iqr) &
            (confs <= q3 + 1.5 * iqr)
        ]

        resultado.append({
            "classe": classe,
            "confianca": float(np.median(confs_validas)),
            "amostras": len(confs_validas)
        })

    return resultado


class BotImagem(BotTelegram):

    def __init__(self):
        super().__init__()
        # Carrega o modelo YOLO treinado (o arquivo "best.pt" deve estar no projeto ou no Colab):
        # self._modelo = YOLO("/content/drive/MyDrive/grp6_bot_telegram/best.pt")

    async def tratar_imagem(self, update, context):

      # Envia uma mensagem inicial para o usuário
      status = await update.message.reply_text(
          "Imagem recebida!\nIniciando análise..."
      )

      # O Telegram envia várias resoluções da imagem (pegamos a maior)
      foto = await update.message.photo[-1].get_file()

      # Salva a imagem em um arquivo temporário
      temp_img = tempfile.NamedTemporaryFile(delete=False, suffix=".jpg").name
      await foto.download_to_drive(temp_img)

      # Executa o YOLO
      resultados = self._modelo.predict(
          source=temp_img,
          conf=0.15,
          verbose=False
      )

      # Remove o arquivo temporário
      os.remove(temp_img)

      # Coleta das detecções
      deteccoes = []

      for r in resultados:
          for box in r.boxes:
              classe_en = r.names[int(box.cls)]
              classe_pt = TRADUCOES.get(classe_en, classe_en)

              deteccoes.append({
                  "classe": classe_pt,
                  "conf": float(box.conf)
              })

      # Se nenhum objeto for detectado
      if not deteccoes:
          await status.edit_text(
              "Poxa, não consegui entender o que está na sua imagem :(\n"
              "Que tal me mandar de outro ângulo?"
          )
          return

      # Análise estatística das detecções
      resultado_final = analise_estatistica_yolo(deteccoes)

      # Resposta final
      resposta = "Show! O que eu vejo na sua imagem se chama...\n\n"

      for d in resultado_final:
          resposta += (
              f"{d['classe'].capitalize()}! Digo isso com "
              f"{d['confianca']:.1%} de confiança!\n"
          )

      # Envia o resultado para o usuário
      await status.edit_text(resposta)
